# Process a directory of TIFFs for batch upload

This will process a book of TIFF files into a directory as-needed for Islandora ingest (Continuing Publications meeting 2019-02-19)

Structure for directory is:

/book/

     1/
       page 1.tif
     2/
       page 2.tif
     n/
       page n.tif
       
Keep "book" directory and have the individual pages in sub-directories in a final 'forIngest'.zip file

In [11]:
# importing & options
import datetime
import logging
import shutil
import sys
import tkinter as tk
from pathlib import Path
from tkinter.filedialog import askdirectory

import pandas as pd
from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

# set Logging Configuration with current level at INFO
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

# set display at 95% width
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:95% !important; }</style>'))

In [12]:
def create_subdirectories_for_ingest(book_directory, file_extension='.tif'):
    
    book_directory_path = Path(book_directory)
    print(f'Processing book at {book_directory_path}')
    
    # get sorted list of all image paths with file_extension
    image_paths_list = sorted([x for x in book_directory_path.iterdir() if str(x).endswith(file_extension)])
    print(f'There are {len(image_paths_list)} "{file_extension}"s in "{book_directory_path}"')
    
    # set ingest stub to add to directory name
    ingest_stub = 'CreatedForIslandoraIngest'
    # get today's date in YYY-MM-DD format
    todays_date = datetime.datetime.now().strftime('%Y-%m-%d')
    # add today's date to ingest stub
    ingest_stub = f'{ingest_stub}_{todays_date}'
    
    # set ingest directory name
    ingest_directory_name = f'{book_directory_path.name}_{ingest_stub}'
    print(f'ingest directory name: {ingest_directory_name}')
    
    # create ingest directory
    ingest_directory_path = book_directory_path.parents[0].joinpath(ingest_directory_name)
    try:
        ingest_directory_path.mkdir()  # existing directory will throw error
        if ingest_directory_path.is_dir():
            print(f'ingest directory path: {ingest_directory_path}')
    except FileExistsError:  # directory already exists
        print(f'********** ingest directory already exists at {ingest_directory_path} **********')

    print(f'To Process: {len(image_paths_list)} images in {book_directory_path}')
    
    # progress bar
    progress_label = Label('Image Being Processed')
    progress_bar = IntProgress(min=0, max=len(image_paths_list))
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    # create a directory for an image then copy image into it
    for index, image_path in enumerate(image_paths_list, start=1):
        
        progress_label.value = image_path.name
        
        # create sub-directory for image
        image_directory_path = ingest_directory_path.joinpath(str(index))
        try:
            image_directory_path.mkdir()  # existing directory will throw error
        except FileExistsError:
            print(f'********** ingest directory already exists at {image_directory_path} **********')
        
        # set copy image path & copy image
        copy_image_path = image_directory_path.joinpath(image_path.name)
        shutil.copy(image_path, copy_image_path)
        
        # set new image name to "page {index}{file_extension}"
        new_image_name = f'page {str(index)}{file_extension}'
        new_image_path = copy_image_path.parents[0].joinpath(new_image_name)
        
        # rename copied image path to new name
        copy_image_path.rename(new_image_path)
        
        progress_bar.value = index
    
    glob_string = f'**/*{file_extension}'
    images_processed_paths_list = list(ingest_directory_path.glob(glob_string))
    print(f'Processed images: {len(images_processed_paths_list)} images in {ingest_directory_path}')

In [17]:
def exit(self):
    self.frame.destroy()
exit_btn=Button(self.frame,text='Exit',command=self.exit,activebackground='grey',activeforeground='#AB78F1',bg='#58F0AB',highlightcolor='red',padx='10px',pady='3px')
exit_btn.place(relx=0.45,rely=0.35)

# https://stackoverflow.com/a/14119223
root = tk.Tk()
root.withdraw()
project_work_path = Path(askdirectory())
root.destroy()

NameError: name 'Button' is not defined

In [14]:
project_work_path

WindowsPath('C:/Users/dlisla/Documents/GitHub/utk_ContinuingPublications/data')

In [5]:
project_work_path = Path(r'Z:\ContinuingPublications\Phoenix\work')
book_directory_paths_list = [x for x in project_work_path.iterdir() if x.is_dir()]
book_directory_paths_list

[WindowsPath('Z:/ContinuingPublications/Phoenix/work/phoenix_2015fall'),
 WindowsPath('Z:/ContinuingPublications/Phoenix/work/phoenix_2016fall'),
 WindowsPath('Z:/ContinuingPublications/Phoenix/work/phoenix_2017spring'),
 WindowsPath('Z:/ContinuingPublications/Phoenix/work/phoenix_2018spring')]

In [6]:
for book_directory_path in book_directory_paths_list:
    
    create_subdirectories_for_ingest(book_directory_path)


Processing book at Z:\ContinuingPublications\Phoenix\work\phoenix_2015fall
There are 33 ".tif"s in "Z:\ContinuingPublications\Phoenix\work\phoenix_2015fall"
ingest directory name: phoenix_2015fall_CreatedForIslandoraIngest_2019-03-06
ingest directory path: Z:\ContinuingPublications\Phoenix\work\phoenix_2015fall_CreatedForIslandoraIngest_2019-03-06
To Process: 33 images in Z:\ContinuingPublications\Phoenix\work\phoenix_2015fall


KeyboardInterrupt: 

In [7]:
book_directory_paths_list[0]

WindowsPath('Z:/ContinuingPublications/Phoenix/work/phoenix_2015fall')

In [9]:
dir = Path('Z:/ContinuingPublications/Phoenix/work/phoenix_2015fall_CreatedForIslandoraIngest_2019-03-06/')

In [11]:
import shutil